# react代理的应用

## 基于通义千问原生接口的ReAct代理实现

* 具体的实现方法见课件《9-react代理的国内大模型实现》

In [10]:
import requests
import json
from langchain.utilities import PythonREPL
from datetime import datetime
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.output_parsers import JsonOutputParser
from dashscope import Generation
import dashscope
from langchain.tools.render import render_text_description
from operator import itemgetter
from langchain_core.tools import tool
import pandas as pd
from bs4 import BeautifulSoup
 
dashscope.api_key = ""

In [11]:
class react_tongyi():

    def __init__(self, tool_list):
        self.tool_list=tool_list
        self.tools = [convert_to_openai_tool(i) for i in tool_list]
        self.tool_name=[i["function"]["name"] for i in self.tools]
        self.parser = JsonOutputParser()

    def prompt_ty(self,content):
        system_prompt_t=f"""Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of {self.tool_name}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
        
Question: {content}
Thought:"""
        prompt=[{"role":"system","content":"Answer the following questions as best you can. You have access to the following tools"}]
        prompt.append({"role":"user","content":system_prompt_t})
        return prompt
    
    def get_response_t(self,messages):
        response = Generation.call(
            model='qwen-turbo',
            messages=messages,
            tools=self.tools,
            result_format='message', # 将输出设置为message形式
        )
        return response
    
    def parser_content(self,out_content):
        return {"name":out_content.split("\nAction: ")[1].split("\nAction")[0],"arguments": self.parser.parse(out_content.split("Input: ")[1])}
    
    def tool_chain(self,model_output):
        tool_map = {tool.name: tool for tool in self.tool_list}
        chosen_tool = tool_map[model_output["name"]]
        return itemgetter("arguments") | chosen_tool
    
    def invoke(self,input_p):
        prompt=self.prompt_ty(input_p)

        for i in range(0,5):
            res=self.get_response_t(prompt)
            res_content=res.output.choices[0].message["content"]
            print(res_content)
            if res_content.find("\nAction: ") != -1 :
                tool_args=self.parser_content(res_content)
                tool_out=self.tool_chain(tool_args)
                prompt[1]["content"]=prompt[1]["content"]+res_content+"\nObservation: "+str(tool_out.invoke(tool_args))+"\nThought:"
            else:
                prompt[1]["content"]=prompt[1]["content"]+res_content
                break

        return(prompt[1]["content"])

In [12]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """将两个整数相乘。"""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    "将两个整数相加。"
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    "对底数求指数幂。"
    return base**exponent

In [13]:
tools = [multiply,add,exponentiate]
chuxing=react_tongyi(tools)

In [19]:
res=chuxing.invoke("5加7乘以6,然后再求结果的2次幂")

 我们需要先执行乘法，然后加法，最后对结果求2次幂。这涉及到的操作是先乘法，后加法，再指数运算。我会使用`multiply`和`add`工具来完成前两步，然后使用`exponentiate`工具计算最终结果。
Action: multiply
Action Input: {"first_int": 7, "second_int": 6}
乘法的结果是42。接下来，我需要将这个结果与5相加，然后对总和求2次幂。
Action: add
Action Input: {"first_int": 5, "second_int": 42}
加法后的结果是47。现在，我需要对47求2次幂。
Action: exponentiate
Action Input: {"base": 47, "exponent": 2}
最终答案是2209。
Final Answer: 2209.


In [20]:
print(res)

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of ['multiply', 'add', 'exponentiate']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
        
Question: 5加7乘以6,然后再求结果的2次幂
Thought: 我们需要先执行乘法，然后加法，最后对结果求2次幂。这涉及到的操作是先乘法，后加法，再指数运算。我会使用`multiply`和`add`工具来完成前两步，然后使用`exponentiate`工具计算最终结果。
Action: multiply
Action Input: {"first_int": 7, "second_int": 6}
Observation: 42
Thought:乘法的结果是42。接下来，我需要将这个结果与5相加，然后对总和求2次幂。
Action: add
Action Input: {"first_int": 5, "second_int": 42}
Observation: 47
Thought:加法后的结果是47。现在，我需要对47求2次幂。
Action: exponentiate
Action Input: {"base": 47, "exponent": 2}
Observation: 2209
Thought:最终答案是2209。
Final Answer: 2209.


### 基于ReAct代理的出行顾问的实现

* 天气与农历工具的实现见课件<8-自定义工具及其应用>部分

In [21]:
@tool
def tianqi(city: str) -> dict:
    """查询最近几日的天气情况，包括温度，天气，湿度，风向等"""
    headers ={"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://apis.juhe.cn/simpleWeather/query"
    params = {
        "key":"", # 在个人中心->我的数据,接口名称上方查看
        "city":city, # 要查询的城市名称或城市ID
    }
    resp = requests.get(url,params,headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json

@tool
def wannianli(date: str) -> dict:
    "查询指定日期（日期格式如：'2024-04-08'）的信息，包括农历，星期几，假期，生肖，习俗，忌讳等"
    date=date.replace("-0","-")
    print(date)
    headers ={"Content-Type": "application/x-www-form-urlencoded"}
    url = "http://v.juhe.cn/calendar/day"
    params = {
        "key":"", # 在个人中心->我的数据,接口名称上方查看
        "date":date, # 指定日期,格式为YYYY-MM-DD,如月份和日期小于10,则取个位,如:2012-1-1
    }
    resp = requests.get(url,params,headers=headers)
    resp_json = json.loads(resp.text)
    return resp_json

@tool
def get_date() -> str:
    "获取当天的日期"
    return datetime.now().date().strftime('%Y-%m-%d')

In [22]:
wannianli.invoke({"date":'2024-04-08'})

2024-4-8


{'reason': 'Success',
 'result': {'data': {'animalsYear': '龙',
   'weekday': '星期一',
   'lunarYear': '甲辰年',
   'lunar': '二月三十',
   'year-month': '2024-4',
   'date': '2024-4-8',
   'suit': '结婚.出行.搬家.签订合同.交易.搬新房.开业.栽种.安床.挂匾.拆卸.出火.收养子女.开光',
   'avoid': '祈福.安葬.祭祀.作灶.入殓.探病',
   'holiday': '',
   'desc': ''}},
 'error_code': 0}

In [23]:
tianqi.invoke("南京")

{'reason': '查询成功!',
 'result': {'city': '南京',
  'realtime': {'temperature': '23',
   'humidity': '48',
   'info': '晴',
   'wid': '00',
   'direct': '东北风',
   'power': '2级',
   'aqi': '30'},
  'future': [{'date': '2024-05-28',
    'temperature': '14/27℃',
    'weather': '晴转多云',
    'wid': {'day': '00', 'night': '01'},
    'direct': '东风'},
   {'date': '2024-05-29',
    'temperature': '18/27℃',
    'weather': '多云转阴',
    'wid': {'day': '01', 'night': '02'},
    'direct': '东南风'},
   {'date': '2024-05-30',
    'temperature': '18/22℃',
    'weather': '雷阵雨转中雨',
    'wid': {'day': '04', 'night': '08'},
    'direct': '西北风转东北风'},
   {'date': '2024-05-31',
    'temperature': '20/30℃',
    'weather': '晴转多云',
    'wid': {'day': '00', 'night': '01'},
    'direct': '东南风转东风'},
   {'date': '2024-06-01',
    'temperature': '19/27℃',
    'weather': '多云转阴',
    'wid': {'day': '01', 'night': '02'},
    'direct': '东风'}]},
 'error_code': 0}

In [24]:
tools = [get_date, tianqi, wannianli]
chuxing=react_tongyi(tools)

In [26]:
res=chuxing.invoke("明天农历几号？")

 需要查询明天的农历日期
Action: wannianli
Action Input: {"date": "tomorrow"}
tomorrow
 看来直接查询明天的农历日期不行，我需要先获取当前日期，然后加一天得到明天的日期，再进行查询。
Action: get_date
Action Input: {}
知道了今天的日期后，我可以推算出明天的农历日期。
Action: wannianli
Action Input: {"date": "2024-05-29"}
2024-5-29
明天的农历日期是2024年四月廿二。
Final Answer: 明天的农历日期是四月廿二。


## 基于爬虫的百度搜索工具的实现与应用

https://zhuanlan.zhihu.com/p/479674917

In [27]:
@tool
def baidu_search(keyword : str) -> str:
    "通过搜索引擎查询指定关键词的相关资料"
    headers = {
		"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
		"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
		"Accept-Language": "zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7",
		"Connection": "keep-alive",
		"Accept-Encoding": "gzip, deflate",
		"Host": "www.baidu.com",
		# 需要更换Cookie
		"Cookie": "BIDUPSID=4EA09413AA348579EBC32BB4171DA6C4; PSTM=1685953325; BAIDUID=4EA09413AA34857921D22606F3465DE3:SL=0:NR=20:FG=1; BD_UPN=12314753; MCITY=-%3A; BDUSS=d4V1kzMVE5TC02V0dzSzBHR0pPZzZjY2xXZ05-YkxPSlFYOTlqVWtlekUxVjFtSVFBQUFBJCQAAAAAAAAAAAEAAAA3oAABd2FuZ3BlbmdlAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMRINmbESDZmRX; BDUSS_BFESS=d4V1kzMVE5TC02V0dzSzBHR0pPZzZjY2xXZ05-YkxPSlFYOTlqVWtlekUxVjFtSVFBQUFBJCQAAAAAAAAAAAEAAAA3oAABd2FuZ3BlbmdlAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAMRINmbESDZmRX; H_PS_PSSID=60269_60278_60282_60287_60296_60253; H_WISE_SIDS=60269_60278_60282_60287_60296_60253; H_WISE_SIDS_BFESS=60269_60278_60282_60287_60296_60253; BAIDUID_BFESS=4EA09413AA34857921D22606F3465DE3:SL=0:NR=20:FG=1; BD_CK_SAM=1; PSINO=3; delPer=0; BA_HECTOR=810l01ak80aha1a101ah0401evoeh21j58flh1v; ZFY=65gS58gkPrLw3aFizT:BRHNeW:AN:A3yhdtUx81zQ4:Aky8:C; shifen[389215454364_77942]=1716797107; BCLID=8729107127908649389; BCLID_BFESS=8729107127908649389; BDSFRCVID=43DOJeC62iPknV7toafpMW8kd_IPxeRTH6ao8FEfGw7FPXyfJA7mEG0P2U8g0KuM8DJTogKKKgOTHICF_2uxOjjg8UtVJeC6EG0Ptf8g0f5; BDSFRCVID_BFESS=43DOJeC62iPknV7toafpMW8kd_IPxeRTH6ao8FEfGw7FPXyfJA7mEG0P2U8g0KuM8DJTogKKKgOTHICF_2uxOjjg8UtVJeC6EG0Ptf8g0f5; H_BDCLCKID_SF=JnIJoD8hJIK3fP36qRojh-40b2T22jnOM6T9aJ5nJDonDtTJKPbMQftXy4cUqPnbQjQkW-tbQpP-HJ7y3R6xytKN2h6waTcQBm6QKl0MLpQWbb0xyUQY3jDzbxnMBMni52OnapTn3fAKftnOM46JehL3346-35543bRTLnLy5KJtMDFRj5-hDTjyDaR-htRX54ofW-oofK-5sh7_bf--D4FrMUTptb3yKCcj2x0y2bbrOfJ9bq5xy5K_hN7l045xbbvlbUnX5qcNMn6HQT3mDlQbbN3i-xuO3gQnWb3cWhvJ8UbSyfbPBTD02-nBat-OQ6npaJ5nJq5nhMJmb67JD-50exbH55uHtRktVx5; H_BDCLCKID_SF_BFESS=JnIJoD8hJIK3fP36qRojh-40b2T22jnOM6T9aJ5nJDonDtTJKPbMQftXy4cUqPnbQjQkW-tbQpP-HJ7y3R6xytKN2h6waTcQBm6QKl0MLpQWbb0xyUQY3jDzbxnMBMni52OnapTn3fAKftnOM46JehL3346-35543bRTLnLy5KJtMDFRj5-hDTjyDaR-htRX54ofW-oofK-5sh7_bf--D4FrMUTptb3yKCcj2x0y2bbrOfJ9bq5xy5K_hN7l045xbbvlbUnX5qcNMn6HQT3mDlQbbN3i-xuO3gQnWb3cWhvJ8UbSyfbPBTD02-nBat-OQ6npaJ5nJq5nhMJmb67JD-50exbH55uHtRktVx5; BDORZ=B490B5EBF6F3CD402E515D22BCDA1598; B64_BOT=1; sug=3; sugstore=0; ORIGIN=0; bdime=0; H_PS_645EC=7da2HdXe6gMkTCYHhNWjWjBKf%2F91xi1Bc2ju7gIhGU8jSNH4YjL1%2BaTa6lI; baikeVisitId=c252a4ed-daf5-4079-8ca6-d287e0ffca21; COOKIE_SESSION=454_1_5_9_12_17_1_0_3_7_0_4_76645_0_0_0_1716797758_1716797108_1716799282%7C9%23409189_19_1716797105%7C9; BDSVRTM=34"
	}
    url="https://www.baidu.com/s?wd="+keyword
    response=requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    text=""
    for i in soup.find_all("div",class_="result c-container xpath-log new-pmd"):
        try:
            text=text+"tite:"+i.find("a").text+"\n"
            text=text+"content:"+i.find("span",class_="content-right_2s-H4").text+"\n"
        except:
            continue
    return text

In [31]:
tools = [baidu_search]
chuxing=react_tongyi(tools)

In [32]:
res=chuxing.invoke("杭州师范大学现任校长是谁")

 我需要搜索最新的信息来找到杭州师范大学现任校长的名字。
Action: baidu_search
Action Input: {"keyword": "杭州师范大学 现任校长"}
根据搜索结果，我可以确认杭州师范大学的现任校长是尚永丰。他不仅是校长，还可能担任了其他职务，如党委副书记、院士等。

Final Answer: 杭州师范大学的现任校长是尚永丰。他同时可能还担任了其他行政职务，如党委副书记等。


In [33]:
print(res)

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take in order, should be one of ['baidu_search']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!
        
Question: 杭州师范大学现任校长是谁
Thought: 我需要搜索最新的信息来找到杭州师范大学现任校长的名字。
Action: baidu_search
Action Input: {"keyword": "杭州师范大学 现任校长"}
Observation: tite:校长尚永丰在2023级新生开学典礼上的讲话
tite:杭州师范大学学校领导-中国高校库-中国高校之窗
content:杭州师范大学党委书记 郭东风 党委副书记、校长 尚永丰 党委副书记 杨政 党委副书记 应飚 纪委书记 李泽泉 党委委员 胡绍华 姜建忠 党委委员、副校长 张杭君 党委委员、副校长 戴丽...
tite:杭州师范大学校长尚永丰院士为2023级研究生讲授开学第一课
content:”9月11日上午，校长尚永丰院士以“杭州师范大学——过去、现在和未来”为题，围绕现代化大学发展理念和发展要素，科学研究的本质、对待科研的态度以及学术规范等话题，为2023级研究生讲授开学第...
tite:尚永丰-杭州师范大学校长介绍→MAIGOO人物
tite:杭州师范大学校长尚永丰院士在115周年校庆启动仪式上的讲话
tite:奋发有为,超越自我!杭师大新生开学典礼上,校长尚永丰院士...
content:杭

## 基于爬虫的头条新闻搜索工具的实现与应用

In [ ]:
@tool
def toutiao_new_search(keyword : str) -> str:
    "通过新闻搜索查询指定关键词的最新资讯"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
        "Accept-Encoding": "gzip, deflate",
        "Host": "so.toutiao.com",
        # 需要更换Cookie
        "Cookie": "tt_webid=7241099634301961765; utm_source=torchlight_preview_test; ttwid=1%7C2cEYeizFUPDRTlgVfnpBXDzZbVBqU47ZoqVkOPUNsng%7C1716808003%7C97ffdfaa8832097f7a26c6bc5853d52fbfc05887470e1bd67d1603537751d996; _tea_utm_cache_4916=undefined; _S_DPR=1.25; _S_IPAD=0; s_v_web_id=verify_lwov5wdv_zdHahoml_zBkB_47vC_9Ie1_Q3HsscEwSeEM; __ac_nonce=066547f0500db9ae978bc; __ac_signature=_02B4Z6wo00f01XgfN4gAAIDAVpj6oP1-21l4PzMAADhX37; __ac_referer=https://so.toutiao.com/search?dvpf=pc&source=search_subtab_switch&keyword=%E6%9D%A8%E5%B9%82&pd=baike&action_type=search_subtab_switch&page_num=0&from=baike&cur_tab_title=baike&search_id=; _S_WIN_WH=954_1018"
    }
    url="https://so.toutiao.com/search?dvpf=pc&source=input&keyword="+keyword+"&pd=information&action_type=search_subtab_switch&page_num=0&from=news&cur_tab_title=news"
    response=requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    text=""
    for i in soup.find_all("div",class_="result-content")[1:11]:
        try:
            text=text+"tite:"+i.find("a").text+"\n"
            text=text+"content:"+i.find("span",class_="text-underline-hover").text+"\n"
        except:
            continue
    return text

In [34]:
tools = [baidu_search,toutiao_new_search]
chuxing=react_tongyi(tools)

In [35]:
res=chuxing.invoke("马云现在在哪里？")

 由于马云是中国著名的企业家，他作为阿里巴巴集团的创始人，其个人行踪通常不会公开详细公告。不过，我可以尝试查找最新的新闻报道来了解他的公开活动。
Action: toutiao_new_search
Action Input: {"keyword": "马云 现在 地点"}
 马云最近的行踪似乎在波黑和日本都有出现，特别是在东京，他似乎已经定居，并且参与了一些公共活动，如教学和私人社交。不过，由于名人隐私保护，详细的个人位置信息通常是保密的。因此，我无法提供确切的当前位置，但可以确认他活跃于公共视野中，特别是教育和文化交流领域。

Final Answer: 马云最近的动态显示，他可能在日本东京生活，特别是在教育领域有所涉猎，并且也出现在波黑等地。但具体的位置信息可能会因隐私原因而难以获取。他似乎过着较为低调的生活，专注于个人的新角色和兴趣。


In [93]:
res=chuxing.invoke("byd公司2024年有哪些新型汽车")

 我需要查找比亚迪公司2024年的新款汽车信息，但通常这类信息还未公开发布，所以我可能需要使用新闻搜索来查看最近的相关报道或预测，或者直接在可靠汽车新闻网站上寻找可能的预告。我会先尝试搜索最新的新闻报道。
Action: toutiao_new_search
Action Input: {"keyword": "比亚迪 2024 新车"}
从搜索结果中，我找到了一些关于比亚迪公司2024年新款汽车的信息。以下是一些亮点：

- 比亚迪全新MPV车型，预计搭载DM-i 5.0插电混动技术。
- 海豹06 DM-i将在5月28日正式上市，定位B级车。
- 宋L系列推出了多款车型，包括204马力版本。
- 唐EV荣耀版、唐DM-p荣耀版以及2024款唐DM-p战神版将全系标配云辇-C。
- 秦L预计于2024年5月28日上市，售价12万元左右。
- 海狮6 PHEV被期待挑战丰田RAV4混合动力。
- 比亚迪王朝和海洋网都有新车发布计划。

这些信息表明比亚迪在2024年确实有多款新车计划，涵盖了不同细分市场。不过，具体的车型配置、细节和上市时间可能会有所变化，建议关注比亚迪官方发布的信息以获取最准确的消息。
